In [2]:
import sys
sys.path.append('../../scripts/')
from s3_support import *

# all

In [3]:
# load donors
q = '''select
            org,
            source,
            email,
            date
        from transactions
        where status='A' '''
donors = redshift_query_read(q, schema='production')

In [5]:
print("{:,} unique donors".format(len(donors['email'].unique())))
print("{:,} entries".format(len(donors)))
print("{:,} orgs".format(len(donors['org'].unique())))
print("{} to {}".format(donors['date'].min(), donors['date'].max()))

4,934,537 unique donors
15,480,500 entries
7,904 orgs
2006-05-18 00:00:00 to 2023-05-20 00:00:00


In [6]:
donors.sort_values('date', ascending=True, inplace=True)
donors_first = donors.groupby('email').first().reset_index()

In [14]:
donors_counts = donors.groupby('email')['date'].count().reset_index()
donors_counts.columns = ['donor', 'transactions']

In [27]:
print("average trans / donor")
donors_counts.merge(donors_first, left_on='donor', right_on='email').groupby('source')['transactions'].agg(['mean', 'median']).reset_index()

average trans / donor


,source,mean,median
0,don_form,3.855335,1.0
1,fb,5.660587,1.0
2,givi,2.653584,1.0
3,kiosk,6.707704,1.0
4,mobile,2.841938,1.0
5,mobilevt,1.716089,1.0
6,p2p,1.605699,1.0
7,sms,2.222857,1.0
8,vt,5.113867,1.0


In [26]:
print("Average new donors / org")
sources = donors_first['source'].value_counts().reset_index()
sources.columns = ['source', 'donors']
source_orgs = donors_first.groupby('source')['org'].nunique().reset_index()
sources = sources.merge(source_orgs, on='source')
sources['ratio'] = sources['donors'] / sources['org']
sources

Average new donors / org


,source,donors,org,ratio
0,don_form,2684620,6191,433.632693
1,p2p,1435969,1415,1014.819081
2,mobile,513963,4047,126.998517
3,vt,146232,2561,57.099570
4,kiosk,61205,305,200.672131
5,sms,56525,1418,39.862482
6,mobilevt,20672,710,29.115493
7,givi,14090,860,16.383721
8,fb,1261,226,5.579646


In [25]:
print("Average new donors / org per year")
donors_first['year'] = pd.to_datetime(donors_first['date']).dt.year
annual = donors_first.groupby(['year', 'source'])[['email', 'org']].nunique().reset_index()
annual['ratio'] = annual['email'] / annual['org']
annual.groupby('source')['ratio'].agg(['mean', 'median']).reset_index()

Average new donors / org per year


,source,mean,median
0,don_form,92.171661,104.388536
1,fb,2.675492,2.437500
2,givi,9.533742,9.787702
3,kiosk,57.682006,48.093284
4,mobile,33.457401,29.376712
5,mobilevt,13.029958,12.929577
6,p2p,325.338896,319.278207
7,sms,17.451095,16.327327
8,vt,19.684330,20.339415


# last 2 years

In [31]:
# load donors
q = '''select
            org,
            source,
            email,
            date
        from transactions
        where 
            status='A' and
            date>=dateadd(year, -2, current_date)'''
donors = redshift_query_read(q, schema='production')

In [32]:
print("{:,} unique donors".format(len(donors['email'].unique())))
print("{:,} entries".format(len(donors)))
print("{:,} orgs".format(len(donors['org'].unique())))
print("{} to {}".format(donors['date'].min(), donors['date'].max()))

2,004,765 unique donors
5,696,395 entries
5,027 orgs
2021-05-26 00:00:00 to 2023-05-25 00:00:00


In [33]:
donors.sort_values('date', ascending=True, inplace=True)
donors_first = donors.groupby('email').first().reset_index()

In [34]:
donors_counts = donors.groupby('email')['date'].count().reset_index()
donors_counts.columns = ['donor', 'transactions']

In [35]:
print("average trans / donor")
donors_counts.merge(donors_first, left_on='donor', right_on='email').groupby('source')['transactions'].agg(['mean', 'median']).reset_index()

average trans / donor


,source,mean,median
0,don_form,3.411976,1.0
1,fb,11.442308,4.5
2,givi,2.776505,1.0
3,kiosk,2.745279,1.0
4,mobile,2.831046,1.0
5,mobilevt,1.390853,1.0
6,p2p,1.450527,1.0
7,sms,2.164460,1.0
8,vt,7.258536,1.0


In [36]:
print("Average new donors / org per year")
donors_first['year'] = pd.to_datetime(donors_first['date']).dt.year
annual = donors_first.groupby(['year', 'source'])[['email', 'org']].nunique().reset_index()
annual['ratio'] = annual['email'] / annual['org']
annual.groupby('source')['ratio'].agg(['mean', 'median']).reset_index()

Average new donors / org per year


,source,mean,median
0,don_form,105.844792,127.698759
1,fb,1.393939,1.000000
2,givi,13.649223,13.662651
3,kiosk,90.768236,77.354167
4,mobile,44.352753,47.477605
5,mobilevt,9.961093,10.350365
6,p2p,332.634489,312.997886
7,sms,22.089971,21.639241
8,vt,24.914224,24.283582
